<a href="https://colab.research.google.com/github/hye0ngyun/TIL/blob/main/4_%EC%B6%9C%EB%A0%A5%EC%B8%B5_%EC%86%8C%ED%94%84%ED%8A%B8%EB%A7%A5%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 출력층 설계하기
- 신경망은 분류와 회귀 문제 모두에 이용할 수 있다.
- 회귀: 항등 함수(identify funtion)
- 분류: 소프트맥스 함수(softmax funtion)

# 소프트맥스
$$
y_k = \frac{e^{a_k}}{\sum_{i=0}^{n-1} e^{a_i}}
$$
- 소프트맥스 함수의 분자는 입력 신호 ${a_k}$의 지수함수, 분모는 모든 입력 신호의 지수 함수의 합이다.
- 즉, 입력 신호를 모두 더한 값이 분모고 입력 하나가 분자이므로 각 분자별(클래스별) 영향도(분류기준)를 알 수 있다.
- 예를 들어 소프트맥스를 거친 뒤 가장 높은 값을 가지고 있는 클래스로 예측할 수 있다.

## 소프트맥스 계산 과정 확인

In [6]:
import numpy as np
a = np.array([0.3, 2.9, 4.0])

exp_a = np.exp(a) # 지수함수
print(exp_a)

sum_exp_a = np.sum(exp_a) # 지수함수의 합
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)
# 소프트맥스를 거친 값들의 합은 항상 1
print(y.sum())

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]
1.0


## 소프트맥스 함수 정의

In [7]:
def softmax(a):
  exp_a = np.exp(a)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y

In [26]:
a = np.array([0.3, 2.9, 4.0])
softmax(a)

array([0.01821127, 0.24519181, 0.73659691])

## 소프트맥스 함수 구현 시 주의점
- 지수함수를 이용하므로 입력으로 매우 큰값이 들어올 경우 오버플로 발생 가능

In [18]:
a = np.array([1010, 1000, 990])
print(np.exp(a)) # [inf inf inf]
print(np.exp(a) / np.sum(np.exp(a))) # [nan nan nan]

c = np.max(a) # c = 1010(최댓값)
print(a - c) # [  0, -10, -20]

# 계산 가능
print(np.exp(a-c) / np.sum(np.exp(a-c))) # [9.99954600e-01, 4.53978686e-05, 2.06106005e-09]
# 전체 합은 여전히 1
print(np.sum(np.exp(a-c) / np.sum(np.exp(a-c))))

[inf inf inf]
[nan nan nan]
[  0 -10 -20]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]
1.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


# 개선된 소프트맥스 함수 정의

In [24]:
def softmax(a):
  c = a.max()
  exp_a = np.exp(a - c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y


In [25]:
a = np.array([1010, 1000, 990])
softmax(a)

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

## 소프트맥스 함수의 특징
- 소프트맥스 함수는 앞에서 설명한것과 출력이 0에서 1.0 사이의 실수이며, 출력의 총합은 1이다.
- 출력 총합이 1이 된다는것은 소프트맥스 함수의 출력을 **확률**로 해석할 수 있다.

In [27]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
y

array([0.01821127, 0.24519181, 0.73659691])

- 위 출력결과를 보면 0번 클래스는 1.8%, 1번클래스는 24%, 2번클래스는 73%로 확인할 수 있다.
- 즉, 소프트맥스 함수를 이용해서 문제를 **확률적(통계적)**으로 대응 가능하다.
- 소프트맥스는 항등함수로 적용전과 후 원소의 대소관계는 변하지 않는다. -> **단조함수**이다.
  - 단조함수란 함수 적용전과 후의 대소관계가 동일한 함수를 뜻한다.
- 결과적으로 신경망으로 **분류**할 떄는 출력층의 소프트맥스 함수를 **생략**해도 된다.
- 그러나 신경망을 **학습**시킬 때는 출력층에서 소프트맥스 함수를 **사용**한다.

## 출력층의 뉴런 수
- 출력층의 뉴런 수는 푸는 문제(회귀, 분류 등)에 따라 적절히 정해야한다.
- 분류에서는 분류하고 싶은 클래스의 수로 설정하는것이 일반적이다.
- 예를 들어 mnist라는 숫자 손글씨 예측 문제의 경우 숫자 0~9까지 10개의 뉴런을 출력층의 뉴런 개수로 정한다.